In [1]:
import autograd.numpy as np

import wecopttool as wot

In [2]:
file = "./bem.nc"
data = wot.read_netcdf(file)
f1 = 0.05
nfreq = 50

In [3]:
amplitude = 0.0625  
idx = 6
freq = idx*f1
waves = wot.waves.regular_wave(f1, nfreq, freq, amplitude)

In [23]:
ndof_pto = 1
pto_names = ["PTO_Heave"]
kinematics = np.eye(ndof_pto)

# controller = wot.pto.controller_unstructured
# ncomponents_pto = 2*nfreq + 1
# pto = wot.pto.PTO(ndof_pto, ncomponents_pto, kinematics, controller, pto_names)

controller = wot.pto.controller_pi
ncomponents_pto = 2
pto = wot.pto.PTO(ndof_pto, ncomponents_pto, kinematics, controller, pto_names)

In [24]:
data = wot.add_zerofreq_to_xr(data)
wec = wot.WEC.from_bem(data, f_add={'PTO': pto.force_on_wec})

Linear damping for DOF "HEAVE" has negative or close to zero terms. Shifting up via linear friction of 36.667232716402985 N/(m/s).


In [25]:
scale_x_wec = 1.0
scale_x_opt = 1.0
scale_obj = 1.0
res = wec.solve(waves, pto.mechanical_average_power, pto.nstate,
                scale_x_wec=scale_x_wec, scale_x_opt=scale_x_opt, scale_obj=scale_obj)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -135.0755469894147
            Iterations: 78
            Function evaluations: 128
            Gradient evaluations: 78


In [26]:
w = data.coords['omega'][idx]
Z = ((1j*w)*(data['mass'] + data['added_mass'][0,0,idx])
     + data['radiation_damping'][0,0,idx]
     + (1/(1j*w)) * data['hydrostatic_stiffness']
     + 36.667232716402985
     )

exc_coeff = data['Froude_Krylov_force'] + data['diffraction_force']
F = wot.wave_excitation(exc_coeff, waves)[idx]

power_theoretical = (np.abs(F)**2 / (8 * Z.real)).values[0,0]
power = -res.fun

power, power_theoretical

(135.0755469894147, 135.0755469893718)

In [27]:
xopt = res.x[-pto.nstate:]
omega_wave = waves.omega[idx].data.item()
m = data['mass'] 
w = data['omega']
A = data['added_mass']
B = data['radiation_damping']
K = data['hydrostatic_stiffness']
Bf = 36.667232716402985
Zi = 1j*w*(A+m) + B+Bf + 1/(1j*w)*K
Zic = Zi[idx].conj().data.item()
optimal_gains_expected = -1*Zic.real + 1j * omega_wave * Zic.imag
optimal_gains_expected, xopt[0] + 1j*xopt[1]

((-1041.666474699652+16779.37934565629j),
 (-1041.6664750878858+16779.37934548958j))

array([-1041.66647509, 16779.37934549])